In [1]:
using Revise
using Bilevel

using ForwardDiff

┌ Info: Precompiling Bilevel [9688c538-179f-11e9-3174-495cea6b7f67]
└ @ Base loading.jl:1187


In [241]:
function get_barnesgrad()
    a1 = 75.196
    a3 = 0.12694
    a5 = 1.0345e-5
    a7 = 0.030234
    a9 = 3.5256e-5
    a11 = 0.25645
    a13 = 1.3514e-5
    a15 = -5.2375e-6
    a17 = 7.0e-10
    a19 = -1.6638e-6
    a21 = 0.0005
    a2 = -3.8112
    a4 = -2.0567e-3
    a6 = -6.8306
    a8 = -1.28134e-3
    a10 = -2.266e-7
    a12 = -3.4604e-3
    a14 = -28.106
    a16 = -6.3e-8
    a18 = 3.4054e-4
    a20 = -2.8673
    
    function eval_obj(x::AbstractArray{T}) where T
        x1 = x[1]
        x2 = x[2]
        y1 = x1*x2
        y2 = y1*x1
        y3 = x2^2
        y4 = x1^2

        f = a1 + a2*x1 + a3*y4 + a4*y4*x1 + a5*y4^2 +
            a6*x2 + a7*y1 + a8*x1*y1 + a9*y1*y4 + a10*y2*y4 +
            a11*y3 + a12*x2*y3 + a13*y3^2 + a14/(x2+1) +
            a15*y3*y4 + a16*y1*y4*x2 + a17*y1*y3*y4 + a18*x1*y3 +
            a19*y1*y3 + a20*exp(a21*y1)
        
        f
    end

    function eval_cons(x::AbstractArray{T}) where T
        x1 = x[1]
        x2 = x[2]
        y1 = x1*x2
        y2 = y1*x1
        y3 = x2^2
        y4 = x1^2
        
        c = zeros(T, 3)
        c[1] = 1 - y1/700.0
        c[2] = y4/25.0^2 - x2/5.0
        c[3] = (x1/500.0- 0.11) - (x2/50.0-1)^2
        
        c
    end
    
    fres = DiffResults.GradientResult(zeros(2))
    fcfg = ForwardDiff.GradientConfig(eval_obj, zeros(2))
    gres = DiffResults.JacobianResult(zeros(3), zeros(2))
    gcfg = ForwardDiff.JacobianConfig(eval_cons, zeros(2))
    function solver_fn(x)
        ForwardDiff.gradient!(fres, eval_obj, x, fcfg)
    
        J = eval_obj(x)
        gJ = ForwardDiff.gradient(eval_obj, x)
        
        HJ = ForwardDiff.hessian(eval_obj, x)
        
        ForwardDiff.jacobian!(gres, eval_cons, x, gcfg)
    
        g = DiffResults.value(gres)
        ceq = zeros(0)
        c = g
    
        dgdx = DiffResults.jacobian(gres)
        gceq = zeros(0,2)
        gc = dgdx
    
        J, ceq, c, gJ, gceq, gc, HJ
    end
    
    solver_fn
end

solver_fn = get_barnesgrad()

x0 = [10.0, 10.0]
options = Dict{String, Any}()
options["Derivative option"] = 1
options["Verify level"] = 1
xopt, info = Bilevel.snopt(solver_fn, 0, 3, x0, options)

display(info)
display(xopt)
display(isapprox(xopt[1], 49.5263; atol=1e-4))
display(isapprox(xopt[2], 19.6228; atol=1e-4))

x0 = [0., 0.]
options = Dict{String, Any}()
options["num_fosteps"] = 3
options["num_sosteps"] = 10
options["c"] = 10000.
options["c_fos"] = 10.
options["c_sos"] = 1.
xopt, info = Bilevel.auglag(solver_fn, 0, 3, x0, options)

display(info)
display(xopt)
display(isapprox(xopt[1], 49.5263; atol=1e-4))
display(isapprox(xopt[2], 19.6228; atol=1e-4))

"Finished successfully: optimality conditions satisfied"

2-element Array{Float64,1}:
 49.52630320092698 
 19.622837627545735

true

true

"Finished successfully"

2-element Array{Float64,1}:
 46.18327103780919
 17.14140847461035

false

false